In [ ]:
!pip install git+https://github.com/huggingface/transformers.git -q
!pip install langchain sentence_transformers faiss-gpu chromadb huggingface_hub langchain_community -q
!pip install --upgrade torch datasets accelerate peft bitsandbytes trl -q

Importamos librerías

In [ ]:
import os, json,re
import pandas as pd
from langchain_community.document_loaders import TextLoader
#from InstructorEmbedding import INSTRUCTOR
#from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
#import pickle
from langchain.docstore.document import Document
#import faiss
from langchain.vectorstores import FAISS
#from langchain import FAISS
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
#from tqdm.autonotebook import trange
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
import transformers, accelerate
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM,GenerationConfig,pipeline
from typing import List
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import JinaEmbeddings
from langchain.chains import VectorDBQA, RetrievalQA,ConversationalRetrievalChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain import LLMChain
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
access_token_read = UserSecretsClient().get_secret("HuggingFace")
login(token = access_token_read)

# Preprocesado 

El preprocesado de datos para el retriever lo hago de dos maneras diferentes:
- En la primera divido los abstracts y saco chunks de 1000
- En la segunda cojo todo el abstract entero 

Cargamos los datos del fichero en formato txt

In [ ]:
#loader = TextLoader("/kaggle/input/100000abstracts/1000000.txt")
#pages = loader.load()

#### Prueba para dividir teniendo en cuenta número de tokens y no de palabras.

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained('hkunlp/instructor-large') # initialize the INSTRUCTOR tokenizer

In [ ]:
#splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=512, chunk_overlap=100, separators=["\n"])
#chunks = splitter.split_documents(pages)

--------------------------------------

Dividimos por abstract y descartamos los títulos

In [ ]:
#def split_text_into_chunks(text):
    # Split the text into paragraphs
#    paragraphs = text.split('\n')[:-1] #ultimo \n
#    chunk_pairs = [Document(page_content=paragraphs[i + 1], metadata={"source": paragraphs[i]}) for i in range(0, len(paragraphs), 2)]
#    return chunk_pairs

In [ ]:
#chunks = split_text_into_chunks(pages[0].page_content)

In [ ]:
#print(temas[1])

Subdividir los abstracts

In [ ]:
#splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, separators=["."])
#chunks = splitter.split_documents(temas)

In [ ]:
#chunks[0]

In [ ]:
#chunks[1]

Modelo para crear los embeddings para el vectorStore, también es necesario para el retriever, para poder crear los embeddings de la query

In [ ]:
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)
        self.model.max_seq_length=8000

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return self.model.encode(documents, batch_size=20, show_progress_bar=True)

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0]

Creamos la vectorStore

In [ ]:
#embedding = SentenceTransformerEmbeddings("jinaai/jina-embeddings-v2-base-es")

#db = FAISS.from_documents(chunks, embedding)
#db.save_local("/kaggle/working/faissNoChunks")

Cargamos la vectorStore ya creada

In [ ]:
embedding = SentenceTransformerEmbeddings("jinaai/jina-embeddings-v2-base-es")
db = FAISS.load_local("/kaggle/input/embeddings/faissNoChunks", embedding,allow_dangerous_deserialization=True)

In [ ]:
db.similarity_search_with_score("Practico deporte habitualmente, hace poco un dedo me crujió y ahora me duele",k=3, fetch_k=5)

# Modelo de Lenguaje

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Quantization is a technique used to reduce the memory and computation requirements 
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mistral-7b model, 
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mistral-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 100 # maximum number of new tokens that can be generated by the model
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=pipe)

## Pruebas
Probamos el chatbot por primera vez

In [ ]:
memory = ConversationBufferMemory(return_messages=True,memory_key='chat_history')
retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = ConversationalRetrievalChain.from_llm(llm=LLM,retriever = retriever,memory=memory)

In [ ]:
query = "Me duele el hombro derecho, el dolor aumenta por la noche, no me he dado ningun golpe, qué puede ser?"
response = qa_chain({'question':query,'chat_history':[]})
response['answer']

In [ ]:
query = "Qué me aconsejas para recuperarme lo antes posible?"
response = qa_chain({'question':query,'chat_history':[]})
response['answer']

# MC QA

Vamos a evaluar el chatbot en un dataset multiple choice

#### Preprocesar dataset
Elimino instancias que sean multiple en vez de single choice 

In [ ]:
df = pd.read_csv("/kaggle/input/medmcqa-medical-mcq-dataset/validation.csv")
df

In [ ]:
df = df.drop(df[df['choice_type'] == 'multi'].index)
df = df.drop(df[df['cop'] == 'nan'].index)
len(df)

In [ ]:
df=df.reset_index(drop=True)
df=df.fillna('None')
len(df)


Estructuro y creo un nuevo dataFrame con la información que nos interesa

In [ ]:
myDict={0:'a', 1:'b', 2:'c', 3:'d'}
for i in range(len(df)):
    df.loc[i,'question']= df.loc[i,'question'] + "\n\n### Select an answer between:\n\n" +"a: " + str(df.iloc[i,2]) + "\nb: " + str(df.iloc[i,3]) + "\nc: " + str(df.iloc[i,4]) + "\nd: " + str(df.iloc[i,5])
    #print(myDict[df.iloc[i,2]])
    df.loc[i, 'correctOption']= myDict[df.iloc[i,6]]
df2=df[['question', 'correctOption']]
display(df2)


In [ ]:
#df_partition1 = df2.loc[250:500]

## Inferencia RAG

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 4})


response_schemas = [
    ResponseSchema(name="answer", description="selected choice from a, b, c or d"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


prompt = PromptTemplate(
    template = """
### [INST] Instruction: Answer with one of the following options, return only the selected option. Here is context to help:

{context}

#\n{format_instructions}\n
### QUESTION:
#{question} [/INST]""", 
input_variables=["context", "question"],
partial_variables={"format_instructions": format_instructions},
    
)

 #Create llm chain 
llm_chain = LLMChain(llm=llm, prompt=prompt)

rag_chain = (RunnableParallel( 
 {"context": retriever, "question": RunnablePassthrough()})
    | llm_chain
    #| output_parser
)


In [ ]:
res = rag_chain.batch(df2.loc[:,'question'].tolist())

In [ ]:
f = open("/kaggle/working/restot.txt", "a")

for i in res:
    f.write(i['text'])

f.close()


##### Procesar respuestas

In [ ]:
pattern = r'"answer":\s*"([a-zA-Z])"'

In [ ]:
count=0
for pred,real in zip(res, df2.loc[:, "correctOption"].tolist()):
    pred = re.search(pattern, str(pred['text']))
    if pred is not None:
        pred = pred.group(1)
        if pred==real:
            count+=1
acc = count/len(res)
print(acc)

In [ ]:
f = open("/kaggle/working/acctot.txt", "a")

f.write(acc)

f.close()

## Inferencia sin RAG

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 4})


response_schemas = [
    ResponseSchema(name="answer", description="selected choice from a, b, c or d"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


prompt = PromptTemplate(
    template = """
### [INST] Instruction: Answer with one of the following options, return only the selected option.
\n{format_instructions}\n
### QUESTION:
{question} [/INST]""", 
input_variables=["question"],
partial_variables={"format_instructions": format_instructions},
    
)

# Create llm chain 
llm_chain = LLMChain(llm=llm, prompt=prompt)

only_llm_chain = (RunnableParallel( 
 {"question": RunnablePassthrough()})
    | llm_chain
    #| output_parser
)


In [ ]:
res = only_llm_chain.batch(df2.loc[:,'question'].tolist())

In [ ]:
f = open("/kaggle/working/restotNoRAG.txt", "a")

for i in res:
    f.write(i['text'])

f.close()


In [ ]:
pattern = r'"answer":\s*"([a-zA-Z])"'

In [ ]:
count=0
for pred,real in zip(res, df2.loc[:, "correctOption"].tolist()):
    pred = re.search(pattern, str(pred['text']))
    if pred is not None:
        pred = pred.group(1)
        if pred==real:
            count+=1
acc = count/len(res)
print(acc)

In [ ]:
f = open("/kaggle/working/acctotNoRAG.txt", "a")

f.write(acc)

f.close()

# UI

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 4})
qa = ConversationalRetrievalChain.from_llm(
        llm=LLM, 
        chain_type="stuff", 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )

In [ ]:
query = "Me duele el hombro derecho, el dolor aumenta por la noche, no me he dado ningun golpe, pero practico deporte, qué puede ser?"
response = qa({'question':query,'chat_history':[]})
response['answer']

In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.qa = qa
    

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [ ]:
pn.extension()


In [ ]:
cb = cbfs()

#file_input = pn.widgets.FileInput(accept='.pdf')
#button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

In [ ]:
dashboard